In [ ]:
# # Random forest
# from sklearn.ensemble import RandomForestClassifier
# print('Training RF')
# rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
# rf.fit(X_train, y_train)

# XGBoost
print('Training XGB')
import xgboost as xgb
y_train_xgb = y_train.astype(int)
y_train_xgb[y_train == 5] = 2
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train, y_train_xgb)

# pred_ideal_rf = constructs_predictions(rf, X_test, ideal_df=test_ideal)
pred_xgb = constructs_predictions(xgb_model, X_test, ideal_df=test_ideal)
pred_random = construct_pred_ideal(X_test, test_ideal, np.random.rand(len(X_test)))

In [ ]:
def construct_desire(df, subject = 'prop_id', target = 'click_bool'):
    # aggregate on prop id, average target and rename column
    desire_df = df.groupby(subject)[target].mean().reset_index().rename(columns={target: 'desire_' + target})
    df = df.merge(desire_df, on=subject, how='left')
    return df, desire_df

X_val, desire_df_click = construct_desire(X_val)
X_val, desire_df_book = construct_desire(X_val, target = 'booking_bool')

def merge_and_drop(df, desire_df_click, desire_df_book):
    df = df.merge(desire_df_click, on='prop_id', how='left')
    df = df.merge(desire_df_book, on='prop_id', how='left')
    df.drop(['click_bool', 'booking_bool'], axis=1, inplace=True)
    df['desire_booking_bool'].fillna(0, inplace=True)
    df['desire_click_bool'].fillna(0, inplace=True)
    return df

X_train = merge_and_drop(X_train, desire_df_click, desire_df_book)
X_test = merge_and_drop(X_test, desire_df_click, desire_df_book)
X_val.drop(['click_bool', 'booking_bool'], axis=1, inplace=True)

# X_train = X_train.merge(desire_df_click, on='prop_id', how='left')
# X_train = X_train.merge(desire_df_book, on='prop_id', how='left')

# X_train.drop(['click_bool', 'booking_bool'], axis=1, inplace=True)
# X_train['desire_booking_bool'].fillna(0, inplace=True)
# X_train['desire_click_bool'].fillna(0, inplace=True)